In [ ]:
!pip install neattext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import re
import nltk
import neattext.functions as nfx
import seaborn as sn

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/ML/BooksDataSet.csv')
data.head()

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,NaN,['Locus Award Nominee for Best Young Adult Boo...,6376780,"['3444695', '1921313', '745221', '171994', '93...",96.0,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,2993816,30516,5.09
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,06/21/03,['Bram Stoker Award for Works for Young Reader...,2507623,"['1593642', '637516', '222366', '39573', '14526']",98.0,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,2632233,26923,7.38
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9999999999999,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,07/11/60,"['Pulitzer Prize for Fiction (1961)', 'Audie A...",4501075,"['2363896', '1333153', '573280', '149952', '80...",95.0,"['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...,2269402,23328,NaN
3,1885.Pride_and_Prejudice,Pride and Prejudice,NaN,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,English,9999999999999,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",...,01/28/13,[],2998241,"['1617567', '816659', '373311', '113934', '767...",94.0,"['United Kingdom', 'Derbyshire, England (Unite...",https://i.gr-assets.com/images/S/compressed.ph...,1983116,20452,NaN
4,41865.Twilight,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,English,9780316015844,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",...,10/05/05,"['Georgia Peach Book Award (2007)', 'Buxtehude...",4964519,"['1751460', '1113682', '1008686', '542017', '5...",78.0,"['Forks, Washington (United States)', 'Phoenix...",https://i.gr-assets.com/images/S/compressed.ph...,1459448,14874,2.1


In [ ]:
data = data.drop_duplicates()

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52428 entries, 0 to 52477
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bookId            52428 non-null  object 
 1   title             52428 non-null  object 
 2   series            23445 non-null  object 
 3   author            52428 non-null  object 
 4   rating            52428 non-null  float64
 5   description       51092 non-null  object 
 6   language          48627 non-null  object 
 7   isbn              52428 non-null  object 
 8   genres            52428 non-null  object 
 9   characters        52428 non-null  object 
 10  bookFormat        50955 non-null  object 
 11  edition           4949 non-null   object 
 12  pages             50085 non-null  object 
 13  publisher         48736 non-null  object 
 14  publishDate       51549 non-null  object 
 15  firstPublishDate  31125 non-null  object 
 16  awards            52428 non-null  object

In [ ]:
data['language'].value_counts()

English                                  42635
Arabic                                    1036
Spanish                                    684
French                                     576
German                                     525
                                         ...  
Aromanian; Arumanian; Macedo-Romanian        1
Basque                                       1
Faroese                                      1
Iranian (Other)                              1
Norwegian Nynorsk; Nynorsk, Norwegian        1
Name: language, Length: 81, dtype: int64

In [ ]:
data = data[data["language"] == 'English']

In [ ]:
data.shape

(42635, 25)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42635 entries, 0 to 52477
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bookId            42635 non-null  object 
 1   title             42635 non-null  object 
 2   series            20689 non-null  object 
 3   author            42635 non-null  object 
 4   rating            42635 non-null  float64
 5   description       42126 non-null  object 
 6   language          42635 non-null  object 
 7   isbn              42635 non-null  object 
 8   genres            42635 non-null  object 
 9   characters        42635 non-null  object 
 10  bookFormat        42189 non-null  object 
 11  edition           3853 non-null   object 
 12  pages             41304 non-null  object 
 13  publisher         40189 non-null  object 
 14  publishDate       42264 non-null  object 
 15  firstPublishDate  25917 non-null  object 
 16  awards            42635 non-null  object

In [ ]:
data = data[['title','author' ,'series','genres', 'description']]
data.head()

,title,author,series,genres,description
0,The Hunger Games,Suzanne Collins,The Hunger Games #1,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...
1,Harry Potter and the Order of the Phoenix,"J.K. Rowling, Mary GrandPré (Illustrator)",Harry Potter #5,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...",There is a door at the end of a silent corrido...
2,To Kill a Mockingbird,Harper Lee,To Kill a Mockingbird,"['Classics', 'Fiction', 'Historical Fiction', ...",The unforgettable novel of a childhood in a sl...
3,Pride and Prejudice,"Jane Austen, Anna Quindlen (Introduction)",NaN,"['Classics', 'Fiction', 'Romance', 'Historical...",Alternate cover edition of ISBN 9780679783268S...
4,Twilight,Stephenie Meyer,The Twilight Saga #1,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...",About three things I was absolutely positive.\...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42635 entries, 0 to 52477
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        42635 non-null  object
 1   author       42635 non-null  object
 2   series       20689 non-null  object
 3   genres       42635 non-null  object
 4   description  42126 non-null  object
dtypes: object(5)
memory usage: 2.0+ MB


In [ ]:
data['genres'] = data['genres'].apply(nfx.remove_special_characters)

In [ ]:
data.head()

,title,author,series,genres,description
0,The Hunger Games,Suzanne Collins,The Hunger Games #1,Young Adult Fiction Dystopia Fantasy Science F...,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...
1,Harry Potter and the Order of the Phoenix,"J.K. Rowling, Mary GrandPré (Illustrator)",Harry Potter #5,Fantasy Young Adult Fiction Magic Childrens Ad...,There is a door at the end of a silent corrido...
2,To Kill a Mockingbird,Harper Lee,To Kill a Mockingbird,Classics Fiction Historical Fiction School Lit...,The unforgettable novel of a childhood in a sl...
3,Pride and Prejudice,"Jane Austen, Anna Quindlen (Introduction)",NaN,Classics Fiction Romance Historical Fiction Li...,Alternate cover edition of ISBN 9780679783268S...
4,Twilight,Stephenie Meyer,The Twilight Saga #1,Young Adult Fantasy Romance Vampires Fiction P...,About three things I was absolutely positive.\...


In [ ]:
data["series"] = data["series"].fillna('')

In [ ]:
def combined_features(row):
    return row['title']+" "+ row['series'] + " " +row['author']+" "+row['genres']+" "+str(row['description'])

data['combined_features'] = data.apply(combined_features, axis =1)

In [ ]:
data.head()

,title,author,series,genres,description,combined_features
0,The Hunger Games,Suzanne Collins,The Hunger Games #1,Young Adult Fiction Dystopia Fantasy Science F...,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,The Hunger Games The Hunger Games #1 Suzanne C...
1,Harry Potter and the Order of the Phoenix,"J.K. Rowling, Mary GrandPré (Illustrator)",Harry Potter #5,Fantasy Young Adult Fiction Magic Childrens Ad...,There is a door at the end of a silent corrido...,Harry Potter and the Order of the Phoenix Harr...
2,To Kill a Mockingbird,Harper Lee,To Kill a Mockingbird,Classics Fiction Historical Fiction School Lit...,The unforgettable novel of a childhood in a sl...,To Kill a Mockingbird To Kill a Mockingbird Ha...
3,Pride and Prejudice,"Jane Austen, Anna Quindlen (Introduction)",,Classics Fiction Romance Historical Fiction Li...,Alternate cover edition of ISBN 9780679783268S...,"Pride and Prejudice Jane Austen, Anna Quindle..."
4,Twilight,Stephenie Meyer,The Twilight Saga #1,Young Adult Fantasy Romance Vampires Fiction P...,About three things I was absolutely positive.\...,Twilight The Twilight Saga #1 Stephenie Meyer ...


In [ ]:
data['clean_features'] = data['combined_features'].apply(nfx.remove_stopwords)

data['clean_features'] = data['clean_features'].apply(nfx.remove_special_characters)

data['clean_features'] = data['clean_features'].str.lower()

data.head()

,title,author,series,genres,description,combined_features,clean_features
0,The Hunger Games,Suzanne Collins,The Hunger Games #1,Young Adult Fiction Dystopia Fantasy Science F...,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,The Hunger Games The Hunger Games #1 Suzanne C...,hunger games hunger games 1 suzanne collins yo...
1,Harry Potter and the Order of the Phoenix,"J.K. Rowling, Mary GrandPré (Illustrator)",Harry Potter #5,Fantasy Young Adult Fiction Magic Childrens Ad...,There is a door at the end of a silent corrido...,Harry Potter and the Order of the Phoenix Harr...,harry potter order phoenix harry potter 5 jk r...
2,To Kill a Mockingbird,Harper Lee,To Kill a Mockingbird,Classics Fiction Historical Fiction School Lit...,The unforgettable novel of a childhood in a sl...,To Kill a Mockingbird To Kill a Mockingbird Ha...,kill mockingbird kill mockingbird harper lee c...
3,Pride and Prejudice,"Jane Austen, Anna Quindlen (Introduction)",,Classics Fiction Romance Historical Fiction Li...,Alternate cover edition of ISBN 9780679783268S...,"Pride and Prejudice Jane Austen, Anna Quindle...",pride prejudice jane austen anna quindlen intr...
4,Twilight,Stephenie Meyer,The Twilight Saga #1,Young Adult Fantasy Romance Vampires Fiction P...,About three things I was absolutely positive.\...,Twilight The Twilight Saga #1 Stephenie Meyer ...,twilight twilight saga 1 stephenie meyer young...


In [ ]:
data['clean_features'] =  [re.sub(r'[^a-zA-Z ]+', '', str(x)) for x in data['clean_features']]


In [ ]:
data.head()

,title,author,series,genres,description,combined_features,clean_features
0,The Hunger Games,Suzanne Collins,The Hunger Games #1,Young Adult Fiction Dystopia Fantasy Science F...,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,The Hunger Games The Hunger Games #1 Suzanne C...,hunger games hunger games suzanne collins you...
1,Harry Potter and the Order of the Phoenix,"J.K. Rowling, Mary GrandPré (Illustrator)",Harry Potter #5,Fantasy Young Adult Fiction Magic Childrens Ad...,There is a door at the end of a silent corrido...,Harry Potter and the Order of the Phoenix Harr...,harry potter order phoenix harry potter jk ro...
2,To Kill a Mockingbird,Harper Lee,To Kill a Mockingbird,Classics Fiction Historical Fiction School Lit...,The unforgettable novel of a childhood in a sl...,To Kill a Mockingbird To Kill a Mockingbird Ha...,kill mockingbird kill mockingbird harper lee c...
3,Pride and Prejudice,"Jane Austen, Anna Quindlen (Introduction)",,Classics Fiction Romance Historical Fiction Li...,Alternate cover edition of ISBN 9780679783268S...,"Pride and Prejudice Jane Austen, Anna Quindle...",pride prejudice jane austen anna quindlen intr...
4,Twilight,Stephenie Meyer,The Twilight Saga #1,Young Adult Fantasy Romance Vampires Fiction P...,About three things I was absolutely positive.\...,Twilight The Twilight Saga #1 Stephenie Meyer ...,twilight twilight saga stephenie meyer young ...


In [ ]:
size = data.shape[0]

In [ ]:
size

42635

In [ ]:
df = data.head(21300)

In [ ]:
df.head()

,title,author,series,genres,description,combined_features,clean_features
0,The Hunger Games,Suzanne Collins,The Hunger Games #1,Young Adult Fiction Dystopia Fantasy Science F...,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,The Hunger Games The Hunger Games #1 Suzanne C...,hunger games hunger games suzanne collins you...
1,Harry Potter and the Order of the Phoenix,"J.K. Rowling, Mary GrandPré (Illustrator)",Harry Potter #5,Fantasy Young Adult Fiction Magic Childrens Ad...,There is a door at the end of a silent corrido...,Harry Potter and the Order of the Phoenix Harr...,harry potter order phoenix harry potter jk ro...
2,To Kill a Mockingbird,Harper Lee,To Kill a Mockingbird,Classics Fiction Historical Fiction School Lit...,The unforgettable novel of a childhood in a sl...,To Kill a Mockingbird To Kill a Mockingbird Ha...,kill mockingbird kill mockingbird harper lee c...
3,Pride and Prejudice,"Jane Austen, Anna Quindlen (Introduction)",,Classics Fiction Romance Historical Fiction Li...,Alternate cover edition of ISBN 9780679783268S...,"Pride and Prejudice Jane Austen, Anna Quindle...",pride prejudice jane austen anna quindlen intr...
4,Twilight,Stephenie Meyer,The Twilight Saga #1,Young Adult Fantasy Romance Vampires Fiction P...,About three things I was absolutely positive.\...,Twilight The Twilight Saga #1 Stephenie Meyer ...,twilight twilight saga stephenie meyer young ...


In [ ]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df["clean_features"])

In [ ]:
cosine_sim = cosine_similarity(count_matrix)

In [ ]:
df.head(10)

,title,author,series,genres,description,combined_features,clean_features
0,The Hunger Games,Suzanne Collins,The Hunger Games #1,Young Adult Fiction Dystopia Fantasy Science F...,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,The Hunger Games The Hunger Games #1 Suzanne C...,hunger games hunger games suzanne collins you...
1,Harry Potter and the Order of the Phoenix,"J.K. Rowling, Mary GrandPré (Illustrator)",Harry Potter #5,Fantasy Young Adult Fiction Magic Childrens Ad...,There is a door at the end of a silent corrido...,Harry Potter and the Order of the Phoenix Harr...,harry potter order phoenix harry potter jk ro...
2,To Kill a Mockingbird,Harper Lee,To Kill a Mockingbird,Classics Fiction Historical Fiction School Lit...,The unforgettable novel of a childhood in a sl...,To Kill a Mockingbird To Kill a Mockingbird Ha...,kill mockingbird kill mockingbird harper lee c...
3,Pride and Prejudice,"Jane Austen, Anna Quindlen (Introduction)",,Classics Fiction Romance Historical Fiction Li...,Alternate cover edition of ISBN 9780679783268S...,"Pride and Prejudice Jane Austen, Anna Quindle...",pride prejudice jane austen anna quindlen intr...
4,Twilight,Stephenie Meyer,The Twilight Saga #1,Young Adult Fantasy Romance Vampires Fiction P...,About three things I was absolutely positive.\...,Twilight The Twilight Saga #1 Stephenie Meyer ...,twilight twilight saga stephenie meyer young ...
5,The Book Thief,Markus Zusak (Goodreads Author),,Historical Fiction Fiction Young Adult Histori...,Librarian's note: An alternate cover edition c...,The Book Thief Markus Zusak (Goodreads Author...,book thief markus zusak goodreads author histo...
6,Animal Farm,"George Orwell, Russell Baker (Preface), C.M. W...",,Classics Fiction Dystopia Fantasy Literature P...,Librarian's note: There is an Alternate Cover ...,"Animal Farm George Orwell, Russell Baker (Pre...",animal farm george orwell russell baker prefac...
7,The Chronicles of Narnia,"C.S. Lewis, Pauline Baynes (Illustrator)",The Chronicles of Narnia (Publication Order) #1–7,Fantasy Classics Fiction Young Adult Childrens...,"Journeys to the end of the world, fantastic cr...",The Chronicles of Narnia The Chronicles of Nar...,chronicles narnia chronicles narnia publicatio...
8,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,J.R.R. Tolkien,The Lord of the Rings #0-3,Fantasy Fiction Classics Adventure Science Fic...,"This four-volume, boxed set contains J.R.R. To...",J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,jrr tolkien book boxed set hobbit lord rings l...
9,Gone with the Wind,Margaret Mitchell,,Classics Historical Fiction Fiction Romance Hi...,"Scarlett O'Hara, the beautiful, spoiled daught...",Gone with the Wind Margaret Mitchell Classics...,gone wind margaret mitchell classics historica...


In [ ]:
relevant_book = "The Chronicles of Narnia"
def get_index(title):
  return df[df['title'] == title].index

book_index = get_index(relevant_book)

In [ ]:
book_index[0]

7

In [ ]:
similar_books = list(enumerate(cosine_sim[book_index[0]]))

In [ ]:
sorted_similar_books = sorted(similar_books, key=lambda x:x[1], reverse=True)

In [ ]:
def get_title_from_index(index):
    return df['title'].iloc[index]
i=0
for book in sorted_similar_books:
    print(get_title_from_index(book[0]), sorted_similar_books[i][1])
    i=i+1
    if i>15:
        break

The Chronicles of Narnia 0.9999999999999997
The Last Battle 0.5199451159196805
The Voyage of the Dawn Treader 0.4493228126735572
The Lion, the Witch and the Wardrobe / The Magician's Nephew 0.44577791412086454
The Lion, the Witch and the Wardrobe 0.3966817518790098
The Magician's Nephew / The Lion, The Witch and the Wardrobe / The Horse and His Boy 0.38074465885551206
The Silver Chair 0.3792486458321147
Prince Caspian 0.3730695144760399
The Chronicles of Prydain 0.37052972568318765
The Great Book of Amber 0.3635385987835049
The Magician's Nephew 0.34993448379304026
The Hunters 0.3403960702072403
The First Judgement 0.32455588011159797
The Anvil of Ice 0.31866354361495086
Dragonlance Chronicles 0.3168796766045237
Chosen 0.31625028374862973
